<a href="https://colab.research.google.com/github/Thaleslsilva/DataScience/blob/master/Graficos_Comparativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplos de Gráficos Comparativos

Aqui você encontra exemplos de alguns dos gráficos comparativos estudados no Capítulo 2.

### Pacotes Python Para Manipulação de Dados e Visualização

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Manipulação de Dados
import numpy as np
import pandas as pd

# Plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Thales de Lima Silva" --iversions

## Carregando os Dados

In [ ]:
# Carrega os dados
titanic = pd.read_csv('https://raw.githubusercontent.com/Thaleslsilva/DataScience/master/VisualizacaoDados/Metodos_Visualizacao/dados/titanic-train.csv')

### Bubble Plot
**Objetivo** :  Exibe representação quantitativa destacando a categoria mais ocorrida com o tamanho da bolha.

Quantas pessoas viajaram em cada classe de passageiros do Titanic?

In [ ]:
# Prepara os dados
pclass = titanic['Pclass'].value_counts().to_frame().reset_index().rename(columns = {'index':'Pclass','Pclass':'Count'})
# Figura
fig = go.Figure(data = [go.Scatter(x = pclass['Pclass'], 
                                   y = pclass['Count'], 
                                   mode = 'markers', 
                                   marker = dict(size = pclass['Count']*0.3))])

# Layout
fig.update_layout(title = 'Viajantes Por Classe do Titanic',
                  xaxis_title = "Classe",
                  yaxis_title = "Número de Pessoas")

# Mostra o gráfico
fig.show()

### Bubble Plot Com Gradiente de Cor
**Objetivo** :  Exibe uma representação quantitativa destacando a categoria mais ocorrida com o gradiente de cor da bolha.

Quantas pessoas viajaram no Titanic por faixa etária? 

In [ ]:
# Prepara os dados
titanic = titanic.dropna()

titanic['age_category'] = np.where((titanic['Age'] < 19), "Abaixo de 19 Anos",
                                   np.where((titanic['Age'] > 18) & (titanic['Age'] <= 30), "19-30 Anos",
                                            np.where((titanic['Age'] > 30) & (titanic['Age'] <= 50), "31-50 Anos",
                                                     np.where(titanic['Age'] > 50, "Acima de 50 Anos", "NULL"))))

age = titanic['age_category'].value_counts().to_frame().reset_index().rename(columns = {'index':'age_category','age_category':'Count'})

# Figura
fig = go.Figure(data = [go.Scatter(x = age['age_category'], 
                                   y = age['Count'],
                                   mode = 'markers',
                                   marker = dict(color = age['Count'],
                                                 size = age['Count'],
                                                 showscale = True))])

# Layout
fig.update_layout(title = 'Viajantes do Titanic Por Faixa Etária',
                  xaxis_title = "Faixa Etária",
                  yaxis_title = "Número de Pessoas")

# Gráfico
fig.show()

## Waterfall
**Objetivo** :  Exibir aumento e diminuição de contagens no gráfico em cascata

Apresente um resumo dos casos de Covid nos meses de junho e julho nos Estados Unidos.

In [ ]:
# Carrega os dados
covid = pd.read_csv('https://raw.githubusercontent.com/Thaleslsilva/DataScience/master/VisualizacaoDados/Metodos_Visualizacao/dados/novel-corona-virus-2019-dataset-covid_19_data.csv')

In [ ]:
# Ajusta coluna de data
covid['ObservationDate'] = pd.to_datetime(covid['ObservationDate'])
covid['month'] = covid['ObservationDate'].dt.month_name()

# Prepara os dados
us_confirmed_july = covid[(covid['Country/Region']=='US') & (covid['month']=='July')][['month','Confirmed']].groupby('month').sum().reset_index()['Confirmed']
us_confirmed_june = covid[(covid['Country/Region']=='US') & (covid['month']=='June')][['month','Confirmed']].groupby('month').sum().reset_index()['Confirmed']
us_recovered_july = covid[(covid['Country/Region']=='US') & (covid['month']=='July')][['month','Recovered']].groupby('month').sum().reset_index()['Recovered']
us_recovered_june = covid[(covid['Country/Region']=='US') & (covid['month']=='June')][['month','Recovered']].groupby('month').sum().reset_index()['Recovered']
us_death_july = covid[(covid['Country/Region']=='US') & (covid['month']=='July')][['month','Deaths']].groupby('month').sum().reset_index()['Deaths']
us_death_june = covid[(covid['Country/Region']=='US') & (covid['month']=='June')][['month','Deaths']].groupby('month').sum().reset_index()['Deaths']

# Figura
fig = go.Figure(go.Waterfall(name = "Casos de Covid", 
                             orientation = "v",
                             measure = ["relative", 
                                        "relative", 
                                        "total", 
                                        "relative", 
                                        "relative", 
                                        "total"],
                             x = ["June Confirmed", 
                                  "July Confirmed", 
                                  "Total Confirmed", 
                                  "June Recoverd & Deaths", 
                                  "July Recovered & Deaths", 
                                  "Total Under Observation"],
                             y = [int(us_confirmed_june), 
                                  int(us_confirmed_july), 
                                  0,
                                  -(int(us_death_june) + int(us_recovered_june)), 
                                  -int((us_death_july) + int(us_recovered_july)), 
                                  0],
                             connector = {"line":{"color":"rgb(63, 63, 63)"}},))

# Layout
fig.update_layout(title = "Casos de Covid em Junho e Julho nos EUA", showlegend = True)

# Gráfico
fig.show()

## Waterfall Múltiplo
**Objetivo**: Exibir aumento e diminuição de contagens no gráfico em cascata

Apresente o mesmo gráfico anterior para os EUA e Brasil na mesma área de plotagem.

In [ ]:
# Já temos os dados dos EUA, agora preparamos os dados do Brasil
br_confirmed_july = covid[(covid['Country/Region']=='Brazil')&(covid['month']=='July')][['month','Confirmed']].groupby('month').sum().reset_index()['Confirmed']
br_confirmed_june = covid[(covid['Country/Region']=='Brazil')&(covid['month']=='June')][['month','Confirmed']].groupby('month').sum().reset_index()['Confirmed']
br_recovered_july = covid[(covid['Country/Region']=='Brazil')&(covid['month']=='July')][['month','Recovered']].groupby('month').sum().reset_index()['Recovered']
br_recovered_june = covid[(covid['Country/Region']=='Brazil')&(covid['month']=='June')][['month','Recovered']].groupby('month').sum().reset_index()['Recovered']
br_death_july = covid[(covid['Country/Region']=='Brazil')&(covid['month']=='July')][['month','Deaths']].groupby('month').sum().reset_index()['Deaths']
br_death_june = covid[(covid['Country/Region']=='Brazil')&(covid['month']=='June')][['month','Deaths']].groupby('month').sum().reset_index()['Deaths']

# Figura
fig = go.Figure()

# Adiciona a camada com o waterfall dos EUA
fig.add_trace(go.Waterfall(x = [["June", 
                                 "June", 
                                 "June",
                                 "June", 
                                 "July", 
                                 "July", 
                                 "July",
                                 "July"],
                                ["June Confirmed", 
                                 "June Recovered", 
                                 "June Death", 
                                 "Total Observation", 
                                 "July Confirmed", 
                                 "July Recovered", 
                                 "July Death", 
                                 "Total Observation"]],
                           measure = [ "relative", 
                                      "relative", 
                                      "relative", 
                                      "total", 
                                      "relative", 
                                      "relative", 
                                      "relative", 
                                      "total"],
                           y = [int(us_confirmed_june),
                                -int(us_recovered_june),
                                -int(us_death_june), 
                                None, 
                                int(us_confirmed_july), 
                                -int(us_recovered_july),
                                -int(us_death_july), 
                                None],
                           name = "EUA"))

# Adiciona a camada com o waterfall do Brasil
fig.add_trace(go.Waterfall(x = [["June", 
                                 "June", 
                                 "June",
                                 "June", 
                                 "July", 
                                 "July", 
                                 "July",
                                 "July"],  
                                ["June Confirmed", 
                                 "June Recovered", 
                                 "June Death", 
                                 "Total Observation", 
                                 "July Confirmed", 
                                 "July Recovered", 
                                 "July Death", 
                                 "Total Observation"]],
                           measure = ["relative", 
                                      "relative", 
                                      "relative", 
                                      "total", 
                                      "relative", 
                                      "relative", 
                                      "relative", 
                                      "total"],
                           y = [int(br_confirmed_june),
                                -int(br_recovered_june),
                                -int(br_death_june), 
                                None, 
                                int(br_confirmed_july), 
                                -int(br_recovered_july),
                                -int(br_death_july), 
                                None],
                           name = "Brasil"))

# Layout
fig.update_layout(title = "Casos de Covid em Junho e Julho nos EUA e Brasil",
                  waterfallgroupgap = 0.5,)

# Gráfico
fig.show()

# Fim